## Mise en place du context

In [18]:
from findspark import init
init()
import pyspark
from pyspark.sql.functions import col

Le contexte est récupéré si existant et dans l'autre cas il sera créé.

In [2]:
sc = pyspark.SparkContext.getOrCreate()

## Chargement des données

In [3]:
import os.path as path
from os import makedirs

In [4]:
data_dir = "res"

In [5]:
if not path.isfile("dac.tar.gz"):
    !wget https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz

In [6]:
if not path.isdir(data_dir):
    makedirs(data_dir)

Le fichier [`dac.tar.gz`](https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz) doit se situer dans le même répertoire.

In [7]:
if not path.isfile(data_dir + "/train.txt") or not path.isfile(data_dir + "/test.txt"):
    !tar -xvzf dac.tar.gz -C $data_dir

In [8]:
df = spark.read.csv(data_dir + "/train.txt", header=False, sep="\t")

## Preparation des données

In [10]:
df2 = df.dropna()

In [11]:
print(df.count())
print(df2.count())

45840617
756554


In [14]:
df2.schema

StructType(List(StructField(_c0,StringType,true),StructField(_c1,StringType,true),StructField(_c2,StringType,true),StructField(_c3,StringType,true),StructField(_c4,StringType,true),StructField(_c5,StringType,true),StructField(_c6,StringType,true),StructField(_c7,StringType,true),StructField(_c8,StringType,true),StructField(_c9,StringType,true),StructField(_c10,StringType,true),StructField(_c11,StringType,true),StructField(_c12,StringType,true),StructField(_c13,StringType,true),StructField(_c14,StringType,true),StructField(_c15,StringType,true),StructField(_c16,StringType,true),StructField(_c17,StringType,true),StructField(_c18,StringType,true),StructField(_c19,StringType,true),StructField(_c20,StringType,true),StructField(_c21,StringType,true),StructField(_c22,StringType,true),StructField(_c23,StringType,true),StructField(_c24,StringType,true),StructField(_c25,StringType,true),StructField(_c26,StringType,true),StructField(_c27,StringType,true),StructField(_c28,StringType,true),StructFi

Convertir les colonne 0 à 13 inclu en float

In [24]:
df3 = df2.select(*(col(c).cast("float") if i < 14 else col(c) for i, c in enumerate(df2.columns)))

Afficher les valeurs uniques pour les attributs 14 à 39 inclu

In [26]:
df3.select(*(col(c) for c in df2.columns[14:])).distinct().show() # marche pas

+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|    _c14|    _c15|    _c16|    _c17|    _c18|    _c19|    _c20|    _c21|    _c22|    _c23|    _c24|    _c25|    _c26|    _c27|    _c28|    _c29|    _c30|    _c31|    _c32|    _c33|    _c34|    _c35|    _c36|    _c37|    _c38|    _c39|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|8cf07265|80e26c9b|e9e3d976|a1b718fb|25c83c98|7e0ccccf|16487bff|0b153874|a73ee510|3b08e48b|65819b7a|cf6ad9d8|2e21cea2|07d13a8f|f3635baf|e2daea07|e5ba7672|f54016b9|21ddcdc9|b1252a9d|7ba7b82b|ad3062eb|3a171ecb|9195f690|e8b83407|3e0e39d6|
|be589b51|58e67aaf|65f9158e|3308b8bf|25c83c98|7e0ccccf|0

In [27]:
df3.schema

StructType(List(StructField(_c0,FloatType,true),StructField(_c1,FloatType,true),StructField(_c2,FloatType,true),StructField(_c3,FloatType,true),StructField(_c4,FloatType,true),StructField(_c5,FloatType,true),StructField(_c6,FloatType,true),StructField(_c7,FloatType,true),StructField(_c8,FloatType,true),StructField(_c9,FloatType,true),StructField(_c10,FloatType,true),StructField(_c11,FloatType,true),StructField(_c12,FloatType,true),StructField(_c13,FloatType,true),StructField(_c14,StringType,true),StructField(_c15,StringType,true),StructField(_c16,StringType,true),StructField(_c17,StringType,true),StructField(_c18,StringType,true),StructField(_c19,StringType,true),StructField(_c20,StringType,true),StructField(_c21,StringType,true),StructField(_c22,StringType,true),StructField(_c23,StringType,true),StructField(_c24,StringType,true),StructField(_c25,StringType,true),StructField(_c26,StringType,true),StructField(_c27,StringType,true),StructField(_c28,StringType,true),StructField(_c29,Strin

## Arrêt du context

In [28]:
sc.stop()